In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd '/content/drive/My Drive/Colab Notebooks/mini project 4/Sentence Pair Classification/abcnn'

/content/drive/My Drive/Colab Notebooks/mini project 4/Sentence Pair Classification/abcnn


In [3]:
%pwd

'/content/drive/My Drive/Colab Notebooks/mini project 4/Sentence Pair Classification/abcnn'

# Data Preparation


In [0]:
%pycat data_prepare.py

In [0]:
%%writefile data_prepare.py
import re
import jieba
import random
import csv
from tensorflow.contrib import learn


class Data_Prepare(object):

    def readfile(self, filename):
        texta = []
        textb = []
        tag = []
        # with open(filename) as tsv_f:
        #     reader = csv.reader(tsv_f, delimiter='\t')
        #     for row in reader:
        #         texta.append(self.pre_processing(row[1]))
        #         textb.append(self.pre_processing(row[2]))
        #         tag.append(row[0])
        with open(filename, 'r') as f:
            for line in f.readlines():
                line = line.strip().split("\t")
                texta.append(self.pre_processing(line[1]))
                textb.append(self.pre_processing(line[2]))
                tag.append(line[0])

        # shuffle
        index = [x for x in range(len(texta))]
        random.shuffle(index)
        texta_new = [texta[x] for x in index]
        textb_new = [textb[x] for x in index]
        tag_new = [tag[x] for x in index]

        type = list(set(tag_new))
        dicts = {}
        tags_vec = []
        for x in tag_new:
            if x not in dicts.keys():
                dicts[x] = 1
            else:
                dicts[x] += 1
            temp = [0] * len(type)
            temp[int(x)] = 1
            tags_vec.append(temp)
        print(dicts)
        return texta_new, textb_new, tags_vec

    def pre_processing(self, text):
        text = re.sub('（[^（.]*）', '', text)
        text = ''.join([x for x in text if '\u4e00' <= x <= '\u9fa5'])
        words = ' '.join(jieba.cut(text)).split(" ")
        words = [x for x in ''.join(words)]
        return ' '.join(words)

    def build_vocab(self, sentences, path):
        # lens = [len(sentence.split(" ")) for sentence in sentences]
        # max_length = max(lens)
        # print("max length ",max_length)
        vocab_processor = learn.preprocessing.VocabularyProcessor(12)
        vocab_processor.fit(sentences)
        vocab_processor.save(path)


if __name__ == '__main__':
    data_pre = data_prepare()
    data_pre.readfile('dataset/sent_pair/bq/train.tsv')

Overwriting data_prepare.py


# Model

In [0]:
%pycat abcnn_model_pre.py

In [0]:
import tensorflow as tf
import numpy as np
import math


class ABCNN():
    def __init__(self, is_trainning, s, w, l2_reg, model_type, vocabulary_size, d0=300, di=50,
                 num_classes=2, num_layers=2):
        """
        Implmenentaion of ABCNNs
        (https://arxiv.org/pdf/1512.05193.pdf)

        :param s: sentence length
        :param w: filter width
        :param l2_reg: L2 regularization coefficient
        :param model_type: Type of the network(BCNN, ABCNN1, ABCNN2, ABCNN3).
        :param num_features: The number of pre-set features(not coming from CNN) used in the output layer.
        :param d0: dimensionality of word embedding(default: 300)
        :param di: The number of convolution kernels (default: 50)
        :param num_classes: The number of classes for answers.
        :param num_layers: The number of convolution layers.
        """
        self.is_trainning = is_trainning
        self.text_a = tf.placeholder(tf.int32, shape=[None, s], name="text_a")
        self.text_b = tf.placeholder(tf.int32, shape=[None, s], name="text_b")
        self.y = tf.placeholder(tf.int32, shape=[None, num_classes], name="y")
        # self.features = tf.placeholder(tf.float32, shape=[None, num_features], name="features")

        # embedding层 论文中采用是预训练好的词向量 这里随机初始化一个词典 在训练过程中进行调整
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.vocab_matrix = tf.Variable(tf.truncated_normal(shape=[vocabulary_size, d0],
                                                                stddev=1.0 / math.sqrt(d0)),
                                            name='vacab_matrix')
            self.x1 = tf.nn.embedding_lookup(self.vocab_matrix, self.text_a)
            self.x2 = tf.nn.embedding_lookup(self.vocab_matrix, self.text_b)

        self.x1 = tf.transpose(self.x1, [0, 2, 1])
        self.x2 = tf.transpose(self.x2, [0, 2, 1])

        # zero padding to inputs for wide convolution
        def pad_for_wide_conv(x):
            return tf.pad(x, np.array([[0, 0], [0, 0], [w - 1, w - 1], [0, 0]]), "CONSTANT", name="pad_wide_conv")

        def cos_sim(v1, v2):
            norm1 = tf.sqrt(tf.reduce_sum(tf.square(v1), axis=1))
            norm2 = tf.sqrt(tf.reduce_sum(tf.square(v2), axis=1))
            dot_products = tf.reduce_sum(v1 * v2, axis=1, name="cos_sim")

            return dot_products / (norm1 * norm2)

        def euclidean_score(v1, v2):
            euclidean = tf.sqrt(tf.reduce_sum(tf.square(v1 - v2), axis=1))
            return 1 / (1 + euclidean)

        def make_attention_mat(x1, x2):
            # x1, x2 = [batch, height, width, 1] = [batch, d, s, 1]
            # x2 => [batch, height, 1, width]
            # [batch, width, wdith] = [batch, s, s]

            # 作者论文中提出计算attention的方法 在实际过程中反向传播计算梯度时 容易出现NaN的情况 这里面加以修改
            # euclidean = tf.sqrt(tf.reduce_sum(tf.square(x1 - tf.matrix_transpose(x2)), axis=1))
            # return 1 / (1 + euclidean)

            x1 = tf.transpose(tf.squeeze(x1, [-1]), [0, 2, 1])
            attention = tf.einsum("ijk,ikl->ijl", x1, tf.squeeze(x2, [-1]))
            return attention

        def convolution(name_scope, x, d, reuse):
            with tf.name_scope(name_scope + "-conv"):
                with tf.variable_scope("conv") as scope:
                    conv = tf.contrib.layers.conv2d(
                        inputs=x,
                        num_outputs=di,
                        kernel_size=(d, w),
                        stride=1,
                        padding="VALID",
                        activation_fn=tf.nn.tanh,
                        weights_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                        weights_regularizer=tf.contrib.layers.l2_regularizer(scale=l2_reg),
                        biases_initializer=tf.constant_initializer(1e-04),
                        reuse=reuse,
                        trainable=True,
                        scope=scope
                    )
                    # Weight: [filter_height, filter_width, in_channels, out_channels]
                    # output: [batch, 1, input_width+filter_Width-1, out_channels] == [batch, 1, s+w-1, di]

                    # [batch, di, s+w-1, 1]
                    conv_trans = tf.transpose(conv, [0, 3, 2, 1], name="conv_trans")
                    return conv_trans

        def w_pool(variable_scope, x, attention):
            # x: [batch, di, s+w-1, 1]
            # attention: [batch, s+w-1]
            with tf.variable_scope(variable_scope + "-w_pool"):
                if model_type == "ABCNN2" or model_type == "ABCNN3":
                    pools = []
                    # [batch, s+w-1] => [batch, 1, s+w-1, 1]
                    attention = tf.transpose(tf.expand_dims(tf.expand_dims(attention, -1), -1), [0, 2, 1, 3])

                    for i in range(s):
                        # [batch, di, w, 1], [batch, 1, w, 1] => [batch, di, 1, 1]
                        pools.append(tf.reduce_sum(x[:, :, i:i + w, :] * attention[:, :, i:i + w, :],
                                                   axis=2,
                                                   keep_dims=True))

                    # [batch, di, s, 1]
                    w_ap = tf.concat(pools, axis=2, name="w_ap")
                else:
                    w_ap = tf.layers.average_pooling2d(
                        inputs=x,
                        # (pool_height, pool_width)
                        pool_size=(1, w),
                        strides=1,
                        padding="VALID",
                        name="w_ap"
                    )
                    # [batch, di, s, 1]

                return w_ap

        def all_pool(variable_scope, x):
            with tf.variable_scope(variable_scope + "-all_pool"):
                if variable_scope.startswith("input"):
                    pool_width = s
                    d = d0
                else:
                    pool_width = s + w - 1
                    d = di

                all_ap = tf.layers.average_pooling2d(
                    inputs=x,
                    # (pool_height, pool_width)
                    pool_size=(1, pool_width),
                    strides=1,
                    padding="VALID",
                    name="all_ap"
                )
                # [batch, di, 1, 1]

                # [batch, di]
                all_ap_reshaped = tf.reshape(all_ap, [-1, d])

                return all_ap_reshaped

        def CNN_layer(variable_scope, x1, x2, d):
            # x1, x2 = [batch, d, s, 1]
            with tf.variable_scope(variable_scope):
                if model_type == "ABCNN1" or model_type == "ABCNN3":
                    with tf.name_scope("att_mat"):
                        aW = tf.get_variable(name="aW",
                                             shape=(s, d),
                                             initializer=tf.contrib.layers.xavier_initializer(),
                                             regularizer=tf.contrib.layers.l2_regularizer(scale=l2_reg))

                        # [batch, s, s]
                        att_mat = make_attention_mat(x1, x2)
                        # att_mat = tf.get_variable('att_mat', [None, x1.get_shape()[2], x1.get_shape()[2]],
                        #             initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float32)

                        # [batch, s, s] * [s,d] => [batch, s, d]
                        # matrix transpose => [batch, d, s]
                        # expand dims => [batch, d, s, 1]
                        x1_a = tf.expand_dims(tf.matrix_transpose(tf.einsum("ijk,kl->ijl", att_mat, aW)), -1)
                        x2_a = tf.expand_dims(tf.matrix_transpose(tf.einsum("ijk,kl->ijl", tf.matrix_transpose(att_mat),
                                                                            aW)), -1)

                        # [batch, d, s, 2]
                        x1 = tf.concat([x1, x1_a], axis=3)
                        x2 = tf.concat([x2, x2_a], axis=3)

                left_conv = convolution(name_scope="left", x=pad_for_wide_conv(x1), d=d, reuse=False)
                right_conv = convolution(name_scope="right", x=pad_for_wide_conv(x2), d=d, reuse=True)

                left_attention, right_attention = None, None

                if model_type == "ABCNN2" or model_type == "ABCNN3":
                    # [batch, s+w-1, s+w-1]
                    att_mat = make_attention_mat(left_conv, right_conv)
                    # [batch, s+w-1], [batch, s+w-1]
                    left_attention, right_attention = tf.reduce_sum(att_mat, axis=2), tf.reduce_sum(att_mat, axis=1)

                left_wp = w_pool(variable_scope="left", x=left_conv, attention=left_attention)
                left_ap = all_pool(variable_scope="left", x=left_conv)
                right_wp = w_pool(variable_scope="right", x=right_conv, attention=right_attention)
                right_ap = all_pool(variable_scope="right", x=right_conv)

                return left_wp, left_ap, right_wp, right_ap

        x1_expanded = tf.expand_dims(self.x1, -1)
        x2_expanded = tf.expand_dims(self.x2, -1)

        self.LO_0 = all_pool(variable_scope="input-left", x=x1_expanded)
        self.RO_0 = all_pool(variable_scope="input-right", x=x2_expanded)

        LI_1, self.LO_1, RI_1, self.RO_1 = CNN_layer(variable_scope="CNN-1", x1=x1_expanded, x2=x2_expanded, d=d0)
        self.sims = [cos_sim(self.LO_0, self.RO_0), cos_sim(self.LO_1, self.RO_1)]

        if num_layers > 1:
            _, LO_2, _, RO_2 = CNN_layer(variable_scope="CNN-2", x1=LI_1, x2=RI_1, d=di)
            self.test = LO_2
            self.test2 = RO_2
            self.sims.append(cos_sim(LO_2, RO_2))

        with tf.variable_scope("output-layer"):
            # self.output_features = tf.concat([self.features, tf.stack(sims, axis=1)], axis=1, name="output_features")
            self.output_features = tf.stack(self.sims, axis=1)

            self.logits = tf.contrib.layers.fully_connected(
                inputs=self.output_features,
                num_outputs=num_classes,
                activation_fn=None,
                weights_initializer=tf.contrib.layers.xavier_initializer(),
                weights_regularizer=tf.contrib.layers.l2_regularizer(scale=l2_reg),
                biases_initializer=tf.constant_initializer(1e-04),
                scope="FC")

        self.score = tf.nn.softmax(self.logits, name='score')
        self.prediction = tf.argmax(self.score, 1, name="prediction")
        self.accuracy = tf.reduce_mean(
            tf.cast(tf.equal(tf.argmax(self.y, axis=1), self.prediction), tf.float32))

        with tf.name_scope('cost'):
            self.cost = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.y, logits=self.logits)
            self.cost = tf.reduce_mean(self.cost)
            weights = [v for v in tf.trainable_variables() if ('w' in v.name) or ('kernel' in v.name)]
            l2_loss = tf.add_n([tf.nn.l2_loss(w) for w in weights]) * 0.01
            self.loss = l2_loss + self.cost

        if not is_trainning:
            return

        tvars = tf.trainable_variables()

        self.gradients = tf.gradients(self.loss, tvars)
        grads, _ = tf.clip_by_global_norm(self.gradients, 10)

        optimizer = tf.train.AdamOptimizer(0.001)
        # grad_check = tf.check_numerics(grads, 'check_numerics caught bad gradients')
        # with tf.control_dependencies([grad_check]):
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))


if __name__ == '__main__':
    abcnn = ABCNN(True, 20, 3, 0.001, 'ABCNN1', vocabulary_size=1000, d0=300, di=50, num_classes=2, num_layers=1)

# Config File

In [0]:
%pycat config.py

In [0]:
class Config(object):

    def __init__(self):
        self.embedding_size = 50  # 词向量维度
        self.hidden_num = 100  # 隐藏层规模
        self.l2_lambda = 0.0
        self.learning_rate = 0.01
        self.dropout_keep_prob = 0.5
        self.attn_size = 200
        self.K = 2

        self.epoch = 10
        self.Batch_Size = 50

# Training Preparation

In [0]:
%pycat train.py

##Training

In [29]:
%%writefile train.py
import tensorflow as tf
import data_prepare
from tensorflow.contrib import learn
import numpy as np
import abcnn_model_pre
import config as config
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn import metrics
import os

con = config.Config()
parent_path = os.path.dirname(os.getcwd())
data_pre = data_prepare.Data_Prepare()


class TrainModel(object):
    '''
        训练模型
        保存模型
    '''
    def pre_processing(self):
        train_texta, train_textb, train_tag = data_pre.readfile(parent_path+'/dataset/sent_pair/bq/train.tsv')
        data = []
        data.extend(train_texta)
        data.extend(train_textb)
        data_pre.build_vocab(data, parent_path+'/save_model/abcnn/vocab.pickle')
        # 加载词典

        print('finished building vocab')


        self.vocab_processor = learn.preprocessing.VocabularyProcessor.restore(parent_path+'/save_model/abcnn/vocab.pickle')

        print('here')
        print(len(train_texta))

        train_texta_embedding = np.array(list(self.vocab_processor.transform(train_texta)))
        train_textb_embedding = np.array(list(self.vocab_processor.transform(train_textb)))

        print('starting dev preprocess')

        dev_texta, dev_textb, dev_tag = data_pre.readfile(parent_path+'/dataset/sent_pair/bq/dev.tsv')
        dev_texta_embedding = np.array(list(self.vocab_processor.transform(dev_texta)))
        dev_textb_embedding = np.array(list(self.vocab_processor.transform(dev_textb)))
        return train_texta_embedding, train_textb_embedding, np.array(train_tag), \
               dev_texta_embedding, dev_textb_embedding, np.array(dev_tag)

    def get_batches(self, texta, textb, tag):
        num_batch = int(len(texta) / con.Batch_Size)
        for i in range(num_batch):
            a = texta[i*con.Batch_Size:(i+1)*con.Batch_Size]
            b = textb[i*con.Batch_Size:(i+1)*con.Batch_Size]
            t = tag[i*con.Batch_Size:(i+1)*con.Batch_Size]
            yield a, b, t

    def trainModel(self, l2, num_l):
        train_texta_embedding, train_textb_embedding, train_tag, \
        dev_texta_embedding, dev_textb_embedding, dev_tag = self.pre_processing()

        # 定义训练用的循环神经网络模型
        # abcnn
        # DEFAULT_CONFIG = [{'type': 'ABCNN-1', 'w': 3, 'n': 50, 'nl': 'tanh'} for _ in range(3)]
        # model = abcnn_mdoel.ABCNN(True, learning_rate=con.learning_rate, conv_layers=1, embed_size=con.embedding_size,
        #                           vocabulary_size=len(self.vocab_processor.vocabulary_),
        #                           sentence_len=len(train_texta_embedding[0]), config=DEFAULT_CONFIG)
        model = abcnn_model_pre.ABCNN(True, len(train_texta_embedding[0]), 3, con.l2_lambda, 'ABCNN3',
                                      vocabulary_size=len(self.vocab_processor.vocabulary_), d0=con.embedding_size,
                                      di=50, num_classes=2, num_layers=num_l)
        print("finished pre processing")
        # 训练模型
        session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            tf.global_variables_initializer().run()
            saver = tf.train.Saver()
            best_f1 = 0.0
            for time in range(con.epoch):
                print("training " + str(time + 1) + ">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
                model.is_trainning = True
                loss_all = []
                accuracy_all = []
                for texta, textb, tag in tqdm(
                        self.get_batches(train_texta_embedding, train_textb_embedding, train_tag)):
                    feed_dict = {
                        model.text_a: texta,
                        model.text_b: textb,
                        model.y: tag
                    }
                    _, cost, accuracy = sess.run([model.train_op, model.loss, model.accuracy], feed_dict)
                    loss_all.append(cost)
                    accuracy_all.append(accuracy)

                print("第" + str((time + 1)) + "次迭代的损失为：" + str(np.mean(np.array(loss_all))) + ";准确率为：" +
                      str(np.mean(np.array(accuracy_all))))

                def dev_step():
                    """
                    Evaluates model on a dev set
                    """
                    loss_all = []
                    accuracy_all = []
                    predictions = []
                    for texta, textb, tag in tqdm(
                            self.get_batches(dev_texta_embedding, dev_textb_embedding, dev_tag)):
                        feed_dict = {
                            model.text_a: texta,
                            model.text_b: textb,
                            model.y: tag
                        }
                        dev_cost, dev_accuracy, prediction = sess.run([model.loss, model.accuracy,
                                                                       model.prediction], feed_dict)
                        loss_all.append(dev_cost)
                        accuracy_all.append(dev_accuracy)
                        predictions.extend(prediction)
                    y_true = [np.nonzero(x)[0][0] for x in dev_tag]
                    y_true = y_true[0:len(loss_all)*con.Batch_Size]
                    f1 = f1_score(np.array(y_true), np.array(predictions), average='weighted')
                    print('分类报告:\n', metrics.classification_report(np.array(y_true), predictions))
                    print("验证集：loss {:g}, acc {:g}, f1 {:g}\n".format(np.mean(np.array(loss_all)),
                                                                      np.mean(np.array(accuracy_all)), f1))
                    return f1

                model.is_trainning = False
                f1 = dev_step()

                if f1 > best_f1:
                    best_f1 = f1
                    saver.save(sess, parent_path + "/save_model/abcnn/model.ckpt")
                    print("Saved model success\n")


if __name__ == '__main__':
    train = TrainModel()
    train.trainModel(0.1, 1)

Overwriting train.py


## l2 reg= 0.01, num layers= 1

In [25]:
!python train.py

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.769 seconds.
Prefix dict has been built succesfully.
{'0': 50000, '1': 50000}
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
finished building vocab
here
100000
starting dev preprocess
{'0': 5000, '1': 5000}



Instructions for updating:
Use keras.layers.AveragePooling2D instead.
Instructions for updating:
Please use `layer.__call__` method instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




## l2 reg = 0.01, num layers 2

In [28]:
!python train.py

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.726 seconds.
Prefix dict has been built succesfully.
{'0': 50000, '1': 50000}
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
finished building vocab
here
100000
starting dev preprocess
{'1': 5000, '0': 5000}



Instructions for updating:
Use keras.layers.AveragePooling2D instead.
Instructions for updating:
Please use `layer.__call__` method instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




##l2 reg = 0.1, num layers 1

In [30]:
!python train.py

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.704 seconds.
Prefix dict has been built succesfully.
{'0': 50000, '1': 50000}
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
finished building vocab
here
100000
starting dev preprocess
{'1': 5000, '0': 5000}



Instructions for updating:
Use keras.layers.AveragePooling2D instead.
Instructions for updating:
Please use `layer.__call__` method instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




In [0]:
!python train.py

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.738 seconds.
Prefix dict has been built succesfully.
{'1': 50000, '0': 50000}
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
finished building vocab
here
100000
starting dev preprocess
{'0': 5000, '1': 5000}



Instructions for updating:
Use keras.layers.AveragePooling2D instead.
Instructions for updating:
Please use `layer.__call__` method instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




# Hyperparameter Model Tuning

We will use grid search.

Checking l2 regularization and number of layers.

l2 regularization: [0, 0.01, 0.1]
number of layers: [1, 2, 3]


In [22]:
from train import TrainModel


l2_reg = [0.0, 0.01, 0.1]
num_lay = [1, 2, 3]


for reg in l2_reg:
    for lay in num_lay:
        print("Testing on l2_reg: ", reg, " and number of layers: ", lay)
        model = TrainModel()
        model.trainModel(reg, lay)



Testing on l2_reg:  0.0  and number of layers:  1
{'1': 50000, '0': 50000}
finished building vocab
here
100000
starting dev preprocess
{'0': 5000, '1': 5000}
INFO:tensorflow:Scale of 0 disables regularizer.


ValueError: ignored

#Results from grid search above:

l2 reg: 0
keep prob: 0.5

l2 reg: 0
keep prob: 0.75

l2 reg: 0
keep prob: 1


l2 reg: .01
keep prob: 0.5

l2 reg: .01
keep prob: 0.75

l2 reg: .01
keep prob: 1


l2 reg: .001
keep prob: 0.5

l2 reg: .001
keep prob: 0.75

l2 reg: .001
keep prob: 1


In [0]:
import tensorflow as tf
print(tf.__version__)

1.15.0
